<a href="https://colab.research.google.com/github/RaimanKrishna/raimankrishna.github.io/blob/main/WK10NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas plotly dash


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [2]:
from google.colab import files
uploaded = files.upload()


Saving Worldbank_Mortality.csv to Worldbank_Mortality.csv


In [3]:
import pandas as pd

# Load the uploaded CSV file
df = pd.read_csv('Worldbank_Mortality.csv')

# Check the column names to understand the structure
print(df.columns)


Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022'],
      dtype='object')


In [4]:
# Melt the dataframe for easier plotting
df_melted = df.melt(id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
                    var_name="Year", value_name="Mortality Rate")

# Convert the 'Year' column to numeric and 'Mortality Rate' to float
df_melted['Year'] = pd.to_numeric(df_melted['Year'], errors='coerce')
df_melted['Mortality Rate'] = pd.to_numeric(df_melted['Mortality Rate'], errors='coerce')

# Drop rows with missing values (if any)
df_melted = df_melted.dropna(subset=['Year', 'Mortality Rate'])

# Check the first few rows to confirm the data is ready
df_melted.head()


,Country Name,Country Code,Indicator Name,Indicator Code,Year,Mortality Rate
2,Afghanistan,AFG,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,1960,354.6
8,United Arab Emirates,ARE,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,1960,200.5
12,Antigua and Barbuda,ATG,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,1960,82.6
13,Australia,AUS,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,1960,24.9
14,Austria,AUT,"Mortality rate, under-5 (per 1,000 live births)",SH.DYN.MORT,1960,42.8


In [5]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd

# Load the data (assuming the data is already preprocessed as df_melted)
df_melted = pd.read_csv('Worldbank_Mortality.csv')
df_melted = df_melted.melt(id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
                           var_name="Year", value_name="Mortality Rate")
df_melted['Year'] = pd.to_numeric(df_melted['Year'], errors='coerce')
df_melted['Mortality Rate'] = pd.to_numeric(df_melted['Mortality Rate'], errors='coerce')
df_melted = df_melted.dropna(subset=['Year', 'Mortality Rate'])

# Create a Dash app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    # Dropdown to select country
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in df_melted['Country Name'].unique()],
        value='India',  # Default value
        style={'width': '50%'}
    ),

    # Slider to select year range
    dcc.RangeSlider(
        id='year-slider',
        min=df_melted['Year'].min(),
        max=df_melted['Year'].max(),
        step=1,
        marks={year: str(year) for year in range(df_melted['Year'].min(), df_melted['Year'].max() + 1, 5)},
        value=[df_melted['Year'].min(), df_melted['Year'].max()]
    ),

    # Graph for plotting the mortality rate
    dcc.Graph(id='mortality-graph')
])

# Callback to update the graph based on user input
@app.callback(
    dash.dependencies.Output('mortality-graph', 'figure'),
    [
        dash.dependencies.Input('country-dropdown', 'value'),
        dash.dependencies.Input('year-slider', 'value')
    ]
)
def update_graph(selected_country, selected_years):
    # Filter data based on selected country and year range
    filtered_df = df_melted[(df_melted['Country Name'] == selected_country) &
                            (df_melted['Year'] >= selected_years[0]) &
                            (df_melted['Year'] <= selected_years[1])]

    # Create a plotly line graph
    fig = px.line(filtered_df, x='Year', y='Mortality Rate', title=f'Mortality Rate (Under 5) in {selected_country}',
                  labels={'Mortality Rate': 'Mortality Rate (per 1,000 live births)', 'Year': 'Year'})

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>